### Task Description:
You might have heard of the 9€ ticket that should be available in Germany from June until August. With this ticket, you are able to take regional trains and busses within Germany for 9€ per month. You're taking the chance and plan a city trip through Germany. Since you don't want to spend too much time in the trains to have more time in the cities, you want to optimize your traveling route. 

Specifically, you want to visit the following cities: **Osnabrück, Münster, Köln, München, Hamburg, Berlin, Hannover**

For task simplicity, we assume that:
1. there are direct connections between all cities.
2. the distance between the cities is a valid indicator for the required travel time.

Of course, both assumptions might not be true in the real world.